# Data-driven Finance: Unobservable Factors & PCA

In classical finance, we often assume certain factors drive stock returns. Here, we relax this assumption and let **data itself discover the factors**.  

We are entering the realm of **unsupervised learning**, where the goal is to find structure in data without pre-labeled outcomes.  

The first step is to collect daily stock prices for a set of stocks (here: Dow Jones tickers) and prepare the data for analysis.



In [20]:
import yfinance as yf
import pandas as pd
import numpy as np


tickers = ['AAPL', 'MSFT', 'JPM', 'GOOGL', 'AMZN', 'NVDA', 'TSLA', 'XOM', 'UNH', 'V']

# Download all data
data_full = yf.download(tickers, start="2020-01-01", end="2025-01-01")

# Extract only the closing prices for each ticker
data = data_full['Close']

# Display the first few rows
print(data.head())


C:\Users\USER\AppData\Local\Temp\ipykernel_19644\513063869.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data_full = yf.download(tickers, start="2020-01-01", end="2025-01-01")
[*********************100%***********************]  10 of 10 completed

Ticker           AAPL       AMZN      GOOGL         JPM        MSFT      NVDA  \
Date                                                                            
2020-01-02  72.538513  94.900497  67.965233  119.573349  152.791122  5.971409   
2020-01-03  71.833275  93.748497  67.609680  117.995438  150.888626  5.875831   
2020-01-06  72.405678  95.143997  69.411774  117.901588  151.278610  5.900472   
2020-01-07  72.065140  95.343002  69.277687  115.897209  149.899307  5.971908   
2020-01-08  73.224411  94.598503  69.770790  116.801292  152.286942  5.983108   

Ticker           TSLA         UNH           V        XOM  
Date                                                      
2020-01-02  28.684000  267.026367  183.549103  54.131077  
2020-01-03  29.534000  264.324127  182.089340  53.695881  
2020-01-06  30.102667  266.159180  181.695587  54.108173  
2020-01-07  31.270666  264.552338  181.215332  53.665337  
2020-01-08  32.809334  270.130310  184.317383  52.856041  


# Compute Daily Log Returns and Standardize

Let $P_t$ be the price at time $t$. Daily **returns** are:

$$
R_t = \frac{P_t - P_{t-1}}{P_{t-1}}
$$

We then compute **log returns** to handle compounding:

$$
r_t = \log(1 + R_t)
$$

Finally, we **standardize** the returns to zero mean and unit variance:

$$
X = \frac{r_t - \bar{r}}{\sigma_r}
$$

This gives us the **data matrix $X$** used for PCA.



In [21]:
returns = data.pct_change().dropna()

log_returns = np.log(1 + returns)



In [30]:
X = (log_returns - log_returns.mean()) / log_returns.std()
print(X.head())

Ticker          AAPL      AMZN     GOOGL       JPM      MSFT      NVDA  \
Date                                                                     
2020-01-03 -0.539248 -0.568605 -0.295657 -0.676133 -0.693405 -0.552076   
2020-01-06  0.348745  0.622869  1.244287 -0.065179  0.092487  0.050678   
2020-01-07 -0.285713  0.062809 -0.134026 -0.865098 -0.518121  0.283506   
2020-01-08  0.751024 -0.375507  0.306532  0.353893  0.780108 -0.017875   
2020-01-09  1.004779  0.181927  0.470071  0.152062  0.603958  0.250546   

Ticker          TSLA       UNH         V       XOM  
Date                                                
2020-01-03  0.642066 -0.564571 -0.479493 -0.396842  
2020-01-06  0.402025  0.340058 -0.147786  0.328875  
2020-01-07  0.852087 -0.346702 -0.175253 -0.403573  
2020-01-08  1.088214  1.078324  0.942967 -0.725660  
2020-01-09 -0.575597 -0.327564  0.369204  0.327839  


# Covariance Matrix of Standardized Returns

Once we have the standardized returns \(X\), we compute the **covariance matrix** \(C\):

$$
C = \frac{X^T X}{N}
$$

Where:

- \(X\) is the standardized returns matrix (size \(N \times P\))
- \(N\) is the number of observations (days)
- \(C\) is a \(P \times P\) matrix representing **pairwise correlations** between stocks

The covariance matrix captures the linear relationships between stocks. A covariance of 0 means no linear correlation, while positive/negative values indicate positive/negative co-movement.

**Note:** Since \(X\) is standardized, the covariance matrix is equivalent to the **correlation matrix**.

This matrix is not diagonal, meaning the stock returns are **correlated**.


In [23]:
C = np.cov(X.T)

# Option 1: Just print the covariance matrix
# print(C)

# Option 2: Convert to DataFrame for nicer display
C_df = pd.DataFrame(C, index=tickers, columns=tickers)
print(C_df)


           AAPL      MSFT       JPM     GOOGL      AMZN      NVDA      TSLA  \
AAPL   1.000000  0.594780  0.652854  0.414321  0.750913  0.614656  0.492477   
MSFT   0.594780  1.000000  0.649973  0.269223  0.680171  0.586771  0.436566   
JPM    0.652854  0.649973  1.000000  0.406429  0.747924  0.600683  0.406230   
GOOGL  0.414321  0.269223  0.406429  1.000000  0.424329  0.332254  0.282216   
AMZN   0.750913  0.680171  0.747924  0.424329  1.000000  0.688373  0.456088   
NVDA   0.614656  0.586771  0.600683  0.332254  0.688373  1.000000  0.476880   
TSLA   0.492477  0.436566  0.406230  0.282216  0.456088  0.476880  1.000000   
XOM    0.422115  0.213523  0.347108  0.457493  0.435818  0.292680  0.198554   
UNH    0.585935  0.409002  0.548355  0.618969  0.609751  0.480090  0.361437   
V      0.288141  0.135746  0.263653  0.572315  0.248586  0.187483  0.152204   

            XOM       UNH         V  
AAPL   0.422115  0.585935  0.288141  
MSFT   0.213523  0.409002  0.135746  
JPM    0.347108 

## Step 3: Linear Transformation with PCA

We transform our standardized returns \(X\) into **uncorrelated components**:

$$
Z = X V
$$

Where:

- \(V \in \mathbb{R}^{P \times P}\) is an **orthogonal matrix** whose columns are the eigenvectors of the covariance matrix \(C\).
- \(Z \in \mathbb{R}^{N \times P}\) is the **encoded data** (principal components).

**Key points:**

1. Using all \(P\) components (\(K = P\)):

$$
\text{Cov}(Z) = V^\top C V = \Lambda
$$

where \(\Lambda\) is diagonal. This means the columns of \(Z\) are **uncorrelated**.

2. Each column of \(V\) corresponds to the **weights of an eigen-portfolio**, simplifying correlated stock analysis.

3. In practice, `sklearn` PCA computes \(V\) automatically as `pca.components_.T`.


In [24]:

# 4️⃣ Eigen decomposition
eigvals, eigvecs = np.linalg.eig(C)

print("Eigenvalues (λ):")
print(eigvals)
print("\nEigenvectors (U):")
print(eigvecs)
print("-" * 50)



Eigenvalues (λ):
[5.15437062 1.46216599 0.70299724 0.62666778 0.19945144 0.30988348
 0.31663159 0.37287482 0.4428116  0.41214544]

Eigenvectors (U):
[[ 0.36889691  0.11900097 -0.07078759 -0.08281578 -0.31270072 -0.63739166
   0.07184211  0.56759727 -0.08053554 -0.00454998]
 [ 0.31876325  0.34336009 -0.05397257  0.29289312 -0.13067054  0.01832165
   0.55039488 -0.40287747 -0.25072604 -0.38245996]
 [ 0.35852272  0.16355659 -0.18025302  0.26764595 -0.2807151   0.32035289
  -0.70386504  0.03774942 -0.02614941 -0.24824071]
 [ 0.28374775 -0.45399224  0.19914035  0.13744624  0.00143336 -0.45027008
  -0.15861398 -0.46851783  0.43473814 -0.13572068]
 [ 0.38577668  0.17027278 -0.21183041  0.04354878  0.86828656 -0.07304374
  -0.07272699  0.10173402  0.01816682  0.01374502]
 [ 0.33484581  0.24920466  0.02673784  0.01837031 -0.16832768  0.05806424
   0.0075703  -0.27839073  0.05066102  0.84432934]
 [ 0.26182106  0.21465672  0.74840269 -0.50595039  0.05849197  0.14051558
  -0.09606641 -0.00629115 -

In [25]:
# 5️⃣ Construct diagonal matrix of eigenvalues
Λ = np.diag(eigvals)
print("Diagonal matrix Λ (of eigenvalues):")
print(Λ)
print("-" * 50)

Diagonal matrix Λ (of eigenvalues):
[[5.15437062 0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         1.46216599 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.70299724 0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.62666778 0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.19945144 0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.30988348
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.31663159 0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.37287482 0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.4428116  0.   

In [43]:
# Sort eigenvalues and eigenvectors
idx = np.argsort(eigvals)[::-1]
eigvals = eigvals[idx]
eigvecs = eigvecs[:, idx]
eigvecs_df = pd.DataFrame(eigvecs, index=tickers, columns=[f'PC{i+1}' for i in range(len(eigvals))])
print("Eigenvalues:\n", eigvecs_df)
print(sum(eigvals))  # should equal number of features (10)

Eigenvalues:
             PC1       PC2       PC3       PC4       PC5       PC6       PC7  \
AAPL   0.368897  0.119001 -0.070788 -0.082816 -0.080536 -0.004550  0.567597   
MSFT   0.318763  0.343360 -0.053973  0.292893 -0.250726 -0.382460 -0.402877   
JPM    0.358523  0.163557 -0.180253  0.267646 -0.026149 -0.248241  0.037749   
GOOGL  0.283748 -0.453992  0.199140  0.137446  0.434738 -0.135721 -0.468518   
AMZN   0.385777  0.170273 -0.211830  0.043549  0.018167  0.013745  0.101734   
NVDA   0.334846  0.249205  0.026738  0.018370  0.050661  0.844329 -0.278391   
TSLA   0.261821  0.214657  0.748403 -0.505950 -0.080475 -0.174268 -0.006291   
XOM    0.251315 -0.357691 -0.476105 -0.646570 -0.301321 -0.048099 -0.239824   
UNH    0.343934 -0.244344 -0.036333 -0.008306  0.548045 -0.036949  0.345646   
V      0.205690 -0.557012  0.293664  0.374818 -0.583480  0.162884  0.172370   

            PC8       PC9      PC10  
AAPL   0.071842 -0.637392 -0.312701  
MSFT   0.550395  0.018322 -0.130671  
JP

In [ ]:
Z = X @ eigvecs
print("First few rows of Z (principal components):")
Z = np.array(X @ eigvecs)   # instead of just X @ eigvecs


print(Z[:5])
print(X.index)
print(type(X.index))



In [44]:
Z_df = pd.DataFrame(Z, index=X.index, columns=[f'PC{i+1}' for i in range(len(eigvals))])
print(Z_df.head())

                 PC1       PC2       PC3       PC4       PC5       PC6  \
Date                                                                     
2020-01-03 -1.350707  0.421521  0.769833 -0.438864 -0.053570 -0.222242   
2020-01-06  1.014996  0.334433 -0.072886  0.183801 -0.648586 -0.523173   
2020-01-07 -0.491170  0.915357  0.676675 -0.486831 -0.185047  0.169135   
2020-01-08  1.393603 -0.000036 -0.129737 -1.535469  0.720468 -0.382842   
2020-01-09  0.918607  0.075947 -0.448596  0.770520  0.107021  0.164186   

                 PC7       PC8       PC9      PC10  
Date                                                
2020-01-03  0.208991 -0.261737  0.321674 -0.086024  
2020-01-06 -0.058660 -0.583802  0.267839 -0.434667  
2020-01-07  0.028791  0.114372  0.538609 -0.315784  
2020-01-08  0.443065 -0.240675  0.001753  0.243702  
2020-01-09  0.701512 -0.011621 -0.470621 -0.003135  


In [15]:
N = Z.shape[0]  # number of samples
C_manual = (Z.T @ Z) / (N - 1)


In [13]:
X_reconstructed = Z @ eigvecs.T


In [14]:
print(X_reconstructed)

                   0         1         2         3         4         5  \
Date                                                                     
2020-01-03 -0.539248 -0.568605 -0.295657 -0.676133 -0.693405 -0.552076   
2020-01-06  0.348745  0.622869  1.244287 -0.065179  0.092487  0.050678   
2020-01-07 -0.285713  0.062809 -0.134026 -0.865098 -0.518121  0.283506   
2020-01-08  0.751024 -0.375507  0.306532  0.353893  0.780108 -0.017875   
2020-01-09  1.004779  0.181927  0.470071  0.152062  0.603958  0.250546   
...              ...       ...       ...       ...       ...       ...   
2024-12-24  0.523050  0.746365  0.330095  0.771660  0.443486  0.043125   
2024-12-26  0.109720 -0.416598 -0.166742  0.141035 -0.186381 -0.134869   
2024-12-27 -0.717821 -0.675774 -0.753527 -0.424247 -0.949455 -0.698995   
2024-12-30 -0.718903 -0.515487 -0.426046 -0.402976 -0.734911  0.030284   
2024-12-31 -0.404498 -0.412105 -0.537311  0.053408 -0.451023 -0.771996   

                   6         7       